####Importing the libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
!pip install xgboost
!pip install lightgbm
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


####Loading the data

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/Data_for_UCI_named.csv")
df

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2.930406,9.487627,2.376523,6.187797,3.343416,-0.658054,-1.449106,-1.236256,0.601709,0.779642,0.813512,0.608385,0.023892,unstable
9996,3.392299,1.274827,2.954947,6.894759,4.349512,-1.663661,-0.952437,-1.733414,0.502079,0.567242,0.285880,0.366120,-0.025803,stable
9997,2.364034,2.842030,8.776391,1.008906,4.299976,-1.380719,-0.943884,-1.975373,0.487838,0.986505,0.149286,0.145984,-0.031810,stable
9998,9.631511,3.994398,2.757071,7.821347,2.514755,-0.966330,-0.649915,-0.898510,0.365246,0.587558,0.889118,0.818391,0.037789,unstable


In [3]:
df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [4]:
df.duplicated().sum()

0

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    10000 non-null  float64
 1   tau2    10000 non-null  float64
 2   tau3    10000 non-null  float64
 3   tau4    10000 non-null  float64
 4   p1      10000 non-null  float64
 5   p2      10000 non-null  float64
 6   p3      10000 non-null  float64
 7   p4      10000 non-null  float64
 8   g1      10000 non-null  float64
 9   g2      10000 non-null  float64
 10  g3      10000 non-null  float64
 11  g4      10000 non-null  float64
 12  stab    10000 non-null  float64
 13  stabf   10000 non-null  object 
dtypes: float64(13), object(1)
memory usage: 1.1+ MB


####Feature encoding

In [6]:
df['stabf'] = df['stabf'].replace({'stable':1, 'unstable':0})

In [7]:
df['stabf'].value_counts()

0    6380
1    3620
Name: stabf, dtype: int64

In [8]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,0
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,1
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,0
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,0
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,0


####Train-test split validation

In [9]:
X = df.drop(['stab', 'stabf'], axis=1)
y = df['stabf']
print(X.shape, y.shape)

(10000, 12) (10000,)


In [10]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(8000, 12) (2000, 12) (8000,) (2000,)


####Feature scaling

In [11]:
scaler = StandardScaler()
scaledX_train = scaler.fit_transform(X_train)
scaledX_test = scaler.transform(X_test)

####XGB Classifier

In [12]:
xgb = XGBClassifier(random_state=1)
xgb.fit(scaledX_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=1, ...)

In [13]:
xgb_pred = xgb.predict(scaledX_test)

In [14]:
score_xgb = accuracy_score(y_test, xgb_pred)
print(f'The accuracy score for the XGB classifier is {score_xgb}')

The accuracy score for the XGB classifier is 0.9455


####Random forest classifier

In [15]:
forest = RandomForestClassifier(random_state=1)
forest.fit(scaledX_train, y_train)
forest_pred = forest.predict(scaledX_test)

In [16]:
score_forest = accuracy_score(y_test, forest_pred)
print(f'The accuracy score for the random forest classifier is {score_forest}')

The accuracy score for the random forest classifier is 0.9295


####LGBM Classifier

In [17]:
light = LGBMClassifier(random_state=1)
light.fit(scaledX_train, y_train)
light_pred = light.predict(scaledX_test)

In [18]:
score_light = accuracy_score(y_test, light_pred)
print(f'The accuarcy score for LGBM classifier is {score_light}')

The accuarcy score for LGBM classifier is 0.9375


####Extra trees classifier

In [19]:
extra = ExtraTreesClassifier(random_state=1)
extra.fit(scaledX_train, y_train)
extra_pred = extra.predict(scaledX_test)

In [20]:
extra_score = accuracy_score(y_test, extra_pred)
print(f'The accuracy score for the Extra trees classifier is {extra_score}')

The accuracy score for the Extra trees classifier is 0.9285


####Hyperparameter Tuning

In [21]:
params ={'n_estimators':[100,300,500,1000], 'min_samples_split':[7,2,5], 'min_samples_leaf':[4,6,8],
         'max_features':['auto','log2', None]}

In [22]:
random = RandomizedSearchCV(estimator=extra, param_distributions=params, cv=5, n_iter=10, scoring='accuracy', n_jobs=-1, 
                            verbose=1, random_state=1)
random.fit(scaledX_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=ExtraTreesClassifier(random_state=1),
                   n_jobs=-1,
                   param_distributions={'max_features': ['auto', 'log2', None],
                                        'min_samples_leaf': [4, 6, 8],
                                        'min_samples_split': [7, 2, 5],
                                        'n_estimators': [100, 300, 500, 1000]},
                   random_state=1, scoring='accuracy', verbose=1)

In [23]:
random.best_params_

{'n_estimators': 500,
 'min_samples_split': 2,
 'min_samples_leaf': 6,
 'max_features': None}

In [24]:
tuned_pred = random.predict(scaledX_test)

In [25]:
tuned_score = accuracy_score(y_test, tuned_pred)
print(f'The accuracy score for the tuned extra trees classifier is {tuned_score}')

The accuracy score for the tuned extra trees classifier is 0.9295


####Feature Importance

In [26]:
importance = list(extra.feature_importances_)

In [27]:
data = list(zip(X_train.columns, importance))

In [28]:
importance_df = pd.DataFrame(data, columns=['features', 'importance']).sort_values(by='importance', ascending=False)
importance_df

,features,importance
1,tau2,0.118445
0,tau1,0.117397
3,tau4,0.115466
2,tau3,0.113169
10,g3,0.096883
11,g4,0.094019
9,g2,0.093676
8,g1,0.089783
6,p3,0.040706
7,p4,0.040579
